In [1]:
import pyspark
from pyspark.sql import SparkSession
app_name = "bubbly"
master = "local[*]"
spark = (SparkSession.builder
    .master(master)
    .config("spark.driver.cores", 1)
    .appName(app_name)
    .getOrCreate() )
sc = spark.sparkContext
print ('SparkContext created')

SparkContext created


In [2]:
# Cargamos el fichero desde hdfs
#accidenteData = spark.read.csv ('hdfs://localhost:9000/user/ubuntu/accidentes/2018.csv',header=True)
#accidenteData = spark.read.csv ('file:///home/ubuntu/Downloads/2018_Accidentalidad.csv',header=True)
accidenteData = spark.read.csv ('file:///home/jose/Descargas/2018.csv',header=True)

In [65]:
import pyspark.sql.functions as func
from pyspark.sql.types import DoubleType

# Reducimos las columnas que tenemos que utilizar mediante la Select
# y aplicamos las funciones necesarias a los datos
accRed=accidenteData\
    .select(
        func.when (\
                (accidenteData['Nº']=='0'),\
                func.regexp_replace (accidenteData['LUGAR ACCIDENTE'], ' - ', ' ESQUINA ')\
            )\
            .otherwise(
                func.func.regexp_replace (accidenteData['LUGAR ACCIDENTE'],' NUM\s*$',' ') + \
                accidenteData['Nº']+', MADRID ESPAÑA'\
            )\
            .alias('direccion') ,\
#        func.when (\
#                (accidenteData['Nº']=='0'),\
#                None\
#            )\
#            .when (\
#                (accidenteData['LUGAR ACCIDENTE'].contains (' KM.')) & \
#                (accidenteData['Nº'].cast(DoubleType()) > 100) ,\
#                accidenteData['Nº'].cast(DoubleType()) / 1000\
#            )\
#            .otherwise(accidenteData['Nº'].cast(DoubleType()))\
#            .alias('numNorm') ,\
        accidenteData['LUGAR ACCIDENTE'].alias('calle'),\
        accidenteData['Nº'].alias('num')\
    )\
    .dropDuplicates ()



### Para geolocalizar las direcciones utilizamos la libreria geocoder de python.
### Se instala con el comando "pip install geocoder"

In [66]:
import geocoder
g=geocoder.osm('calle de dolores barranco num 44, madrid españa')
g.osm

In [67]:
g

<[ERROR - No results found] Osm - Geocode [empty]>

In [69]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
accRed.select ('direccion').toPandas()

,direccion
0,CALLE MAYOR ESQUINA PLAZA DEL COMANDANTE LAS MORENAS
1,CALLE DE LAS MARISMAS ESQUINA CALLE DE MARUJA GARCIA ROMERO
2,None
3,None
4,None
5,None
6,None
7,None
8,None
9,None
